In [1]:
from hashlib import sha256

In [2]:
class block(object):
    def __init__(self, parent, transactions):
        self._parent = parent
        self._transactions = transactions
        self._nonce = None
        self._nonce_len = 8
        self._block_hash = None
        
    def do_work(self, N):
        ''' find nonce that results in block hash with N leading zeros '''
        # init nonce at zero:
        self._nonce = 0
        while not self.check_work(N):
            self._nonce += 1
        self._block_hash = self.calc_block_hash()
    
    def decode_nonce(self, nonce):
        ''' converts int representation of given nonce to str form using lowercase alphabet '''
        K = 26 # number of letters in alphabet
        offset = 97 # where the alphabet starts in utf-8
        xs = [nonce % K]
        r = nonce // K
        while r > 0:
            xs.append(r % K)
            r = r // K
        return bytes(xs).decode('utf-8')
    
    def decode_nonce_utf8(self, nonce):
        return decode_nonce(nonce).encode('utf-8')
        
    def check_work(self, N):
        ''' recalculate block hash, check it has desired number of leading zeros '''
        h = self.calc_block_hash()
        #target_len = len(sha256(''.encode('utf-8')).hexdigest())
        return all([x=='0' for x in list(h[:N])])
    
    def calc_block_hash(self):
        ''' calculate block hash '''
        t = ','.join(self._transactions) + '-' + self._parent.get_block_hash() + '-' + self.decode_nonce(self._nonce)
        return sha256(t.encode('utf-8')).hexdigest()
    
    def get_block_hash(self):
        return self._block_hash
    
    def len_chain(self):
        # count how many ancestors this block has:
        if self._parent is None:
            return 1
        return 1 + self._parent.len_chain()

In [3]:
N = 5 # difficulty in creating proof of work

b0 = block(None, []) # initial hash is a dummy root node
b0._block_hash = sha256(''.encode('utf-8')).hexdigest()

b1 = block(b0, ['help'])
b1.do_work(N)
print('check b1 work:', b1.check_work(N))
print('b1 is of length', b1.len_chain())

b2 = block(b1, ['help', 'yesterday'])
b2.do_work(N)
print('check b2 work:', b2.check_work(N))
print('b2 is of length', b2.len_chain())

b3 = block(b2, ['help', 'yesterday', 'revolution'])
b3.do_work(N)
print('check b3 work:', b3.check_work(N))
print('b3 is of length', b3.len_chain())

check b1 work: True
b1 is of length 2
check b2 work: True
b2 is of length 3
check b3 work: True
b3 is of length 4


In [4]:
[b0.get_block_hash(), b1.get_block_hash(), b2.get_block_hash(), b3.get_block_hash()]

['e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
 '000008d7dde1692654c4fa7a7d4779fe2b4876b827a7f5539892eb4dd78908cd',
 '000002bd1fdf83f9d35ccd30d3abbd290da78f9c7112b0104df0c8c846352aea',
 '00000f8360c2d0337c88629c276be7e66a849de47bf24d96abc8761fc2c1d8ee']

notice the leading zeros for all but the root node!